In [1]:
import pandas as pd
import tensorflow as tf

df_train_test = pd.DataFrame(columns=["Home Team", "Away Team",
                                      "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                                      "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                                      "Points Obtained This Season", "Points Obtained by Opponent This Season",
                                      "Goals Scored This Season", "Goals Conceded by Opponent This Season",
                                      "Goals Scored in Game"])
teams = {}
teams_stats = {}
for i in range(10, 18):
    df = pd.read_csv("./archive-3/Datasets/20" + str(i) + "-" + str(i + 1) + ".csv")
    teams[i] = set(df["HomeTeam"].unique())
    teams_stats[i] = {}
    for team in teams[i]:
        teams_stats[i][team] = [0, 0, 0, 0]
    for index, row in df.iterrows():
        if i > 10 and teams_stats[i][row["HomeTeam"]][3] > 0 and teams_stats[i][row["AwayTeam"]][3]:
            row1 = [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, row["FTHG"]]
            row2 = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, row["FTAG"]]
            if row["HomeTeam"] in teams[i - 1]:
                row1[2] = teams_stats[i - 1][row["HomeTeam"]][0] / 38
                row1[4] = teams_stats[i - 1][row["HomeTeam"]][1] / 38
                row2[3] = teams_stats[i - 1][row["HomeTeam"]][0] / 38
                row2[5] = teams_stats[i - 1][row["HomeTeam"]][2] / 38
            if row["AwayTeam"] in teams[i - 1]:
                row1[3] = teams_stats[i - 1][row["AwayTeam"]][0] / 38
                row1[5] = teams_stats[i - 1][row["AwayTeam"]][2] / 38
                row2[2] = teams_stats[i - 1][row["AwayTeam"]][0] / 38
                row2[4] = teams_stats[i - 1][row["AwayTeam"]][1] / 38
            row1[6] = teams_stats[i][row["HomeTeam"]][0] / teams_stats[i][row["HomeTeam"]][3]
            row1[8] = teams_stats[i][row["HomeTeam"]][1] / teams_stats[i][row["HomeTeam"]][3]
            row2[7] = teams_stats[i][row["HomeTeam"]][0] / teams_stats[i][row["HomeTeam"]][3]
            row2[9] = teams_stats[i][row["HomeTeam"]][2] / teams_stats[i][row["HomeTeam"]][3]
            row1[7] = teams_stats[i][row["AwayTeam"]][0] / teams_stats[i][row["AwayTeam"]][3]
            row1[9] = teams_stats[i][row["AwayTeam"]][2] / teams_stats[i][row["AwayTeam"]][3]
            row2[6] = teams_stats[i][row["AwayTeam"]][0] / teams_stats[i][row["AwayTeam"]][3]
            row2[8] = teams_stats[i][row["AwayTeam"]][1] / teams_stats[i][row["AwayTeam"]][3]
            df_train_test.loc[-1] = row1
            df_train_test.index = df_train_test.index + 1
            df_train_test = df_train_test.sort_index()
            df_train_test.loc[-1] = row2
            df_train_test.index = df_train_test.index + 1
            df_train_test = df_train_test.sort_index()
        if i < 18:
            teams_stats[i][row["HomeTeam"]][3] += 1
            teams_stats[i][row["AwayTeam"]][3] += 1
            teams_stats[i][row["HomeTeam"]][1] += row["FTHG"]
            teams_stats[i][row["HomeTeam"]][2] += row["FTAG"]
            teams_stats[i][row["AwayTeam"]][1] += row["FTAG"]
            teams_stats[i][row["AwayTeam"]][2] += row["FTHG"]
            if row["FTHG"] > row["FTAG"]:
                teams_stats[i][row["HomeTeam"]][0] += 3
            elif row["FTHG"] == row["FTAG"]:
                teams_stats[i][row["HomeTeam"]][0] += 1
                teams_stats[i][row["AwayTeam"]][0] += 1
            else:
                teams_stats[i][row["AwayTeam"]][0] += 3

/Users/muyuanyang/opt/anaconda3/envs/ml/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_train = df_train_test[:-760]

4418
5178


In [3]:
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay

model = models.Sequential()
model.add(layers.Dense(32, input_dim=10, activation='relu', kernel_regularizer=regularizers.l2(l=0.1)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(l=0.1)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(l=0.1)))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(l=0.1)))
model.add(layers.Dense(1, activation='linear'))

initial_learning_rate = 0.01
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)
optimizer = optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_squared_error'])

Metal device set to: Apple M1


2024-02-10 11:41:59.445478: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-10 11:41:59.445741: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
model.fit(df_train[["Home Team", "Away Team",
                    "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                    "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                    "Points Obtained This Season", "Points Obtained by Opponent This Season",
                    "Goals Scored This Season", "Goals Conceded by Opponent This Season"]],
          df_train[["Goals Scored in Game"]], epochs=50, validation_split=0.2)

Epoch 1/50


2024-02-10 11:42:12.022119: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-10 11:42:12.337380: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


111/111 [==============================] - ETA: 0s - loss: 1.9499 - mean_squared_error: 1.7155

2024-02-10 11:42:14.779242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


111/111 [==============================] - 3s 24ms/step - loss: 1.9499 - mean_squared_error: 1.7155 - val_loss: 0.9064 - val_mean_squared_error: 1.5993
Epoch 2/50
111/111 [==============================] - 2s 21ms/step - loss: 0.9462 - mean_squared_error: 1.7141 - val_loss: 0.9198 - val_mean_squared_error: 1.5580
Epoch 3/50
111/111 [==============================] - 2s 21ms/step - loss: 0.9470 - mean_squared_error: 1.7131 - val_loss: 0.9049 - val_mean_squared_error: 1.6039
Epoch 4/50
111/111 [==============================] - 2s 22ms/step - loss: 0.9486 - mean_squared_error: 1.7064 - val_loss: 0.9124 - val_mean_squared_error: 1.5795
Epoch 5/50
111/111 [==============================] - 2s 22ms/step - loss: 0.9452 - mean_squared_error: 1.7117 - val_loss: 0.9094 - val_mean_squared_error: 1.5890
Epoch 6/50
111/111 [==============================] - 2s 22ms/step - loss: 0.9474 - mean_squared_error: 1.7082 - val_loss: 0.9229 - val_mean_squared_error: 1.6392
Epoch 7/50
111/111 [=============

In [5]:
df_test = df_train_test[-760:]
preds = model.predict(df_test[["Home Team", "Away Team",
                               "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                               "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                               "Points Obtained This Season", "Points Obtained by Opponent This Season",
                               "Goals Scored This Season", "Goals Conceded by Opponent This Season"]])

760
24/24 [==============================] - 0s 3ms/step


2024-02-10 11:45:11.524666: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [6]:
model.evaluate(df_test[["Home Team", "Away Team",
                               "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                               "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                               "Points Obtained This Season", "Points Obtained by Opponent This Season",
                               "Goals Scored This Season", "Goals Conceded by Opponent This Season"]],
              df_test[["Goals Scored in Game"]])

24/24 [==============================] - 0s 16ms/step - loss: 0.9491 - mean_squared_error: 1.8110


[0.9490837454795837, 1.81095290184021]